In [1]:
from transformers import AutoImageProcessor, TFSwinModel, TFSwinForImageClassification, SwinConfig
import tensorflow as tf
from datasets import load_dataset
import numpy as np

2023-03-30 11:31:11.348318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 11:31:15.224694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [62]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
inputs = image_processor(image, return_tensors="tf")

No config specified, defaulting to: cats-image/image
Found cached dataset cats-image (/home/deewens/.cache/huggingface/datasets/huggingface___cats-image/image/1.9.0/68fbc793fb10cd165e490867f5d61fa366086ea40c73e549a020103dcb4f597e)


  0%|          | 0/1 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [102]:
inputs

{'pixel_values': <tf.Tensor: shape=(1, 3, 224, 224), dtype=float32, numpy=
 array([[[[ 0.31381115,  0.43368444,  0.4850587 , ..., -0.35405433,
           -0.33692956, -0.35405433],
          [ 0.3651854 ,  0.43368444,  0.46793392, ..., -0.35405433,
           -0.35405433, -0.38830382],
          [ 0.31381115,  0.39943492,  0.41655967, ..., -0.45680285,
           -0.42255333, -0.38830382],
          ...,
          [ 1.9064132 ,  1.7865399 ,  1.6495419 , ...,  1.6152923 ,
            1.4954191 ,  1.4440448 ],
          [ 1.8721637 ,  1.8036647 ,  1.7522904 , ...,  1.4097953 ,
            1.1357993 ,  0.9816765 ],
          [ 1.8721637 ,  1.7180408 ,  1.7351656 , ...,  0.12543888,
           -0.16568205, -0.4739276 ]],
 
         [[-1.6155462 , -1.6155462 , -1.6155462 , ..., -1.7906162 ,
           -1.7906162 , -1.8081232 ],
          [-1.5630252 , -1.5630252 , -1.5630252 , ..., -1.7731092 ,
           -1.7556022 , -1.7731092 ],
          [-1.6330532 , -1.5980392 , -1.5630252 , ..., -1.8

In [60]:
model = TFSwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model.config.num_channels

All model checkpoint layers were used when initializing TFSwinForImageClassification.

Some layers of TFSwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized: ['swin/encoder/layers.2/blocks.2/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.3/attention/self/relative_position_index:0', 'swin/encoder/layers.3/blocks.0/attention/self/relative_position_index:0', 'swin/encoder/layers.1/blocks.0/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.1/attention/self/relative_position_index:0', 'swin/encoder/layers.3/blocks.1/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.4/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.5/attention/self/relative_position_index:0', 'swin/encoder/layers.1/blocks.1/attention/self/relative_position_index:0', 'swin/encoder/layers.0/blocks.0/attention/self/relative_position_index:0', 's

In [12]:
logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = int(tf.math.argmax(logits, axis=-1))
print(model.config.id2label[predicted_label])

No config specified, defaulting to: cats-image/image
Found cached dataset cats-image (/home/deewens/.cache/huggingface/datasets/huggingface___cats-image/image/1.9.0/68fbc793fb10cd165e490867f5d61fa366086ea40c73e549a020103dcb4f597e)


  0%|          | 0/1 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
All model checkpoint layers were used when initializing TFSwinForImageClassification.

Some layers of TFSwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized: ['swin/encoder/layers.2/blocks.2/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.3/attention/self/relative_position_index:0', 'swin/encoder/layers.3/blocks.0/attention/self/relative_position_index:0', 'swin/encoder/layers.1/blocks.0/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.1/attention/self/relative_position_index:0', 'swin/encoder/layers.3/blocks.1/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.4/attention/self/relative_position_index:0', 'swin/encoder/layers.2/blocks.5/attention/self/relative

tabby, tabby cat


In [14]:
logits.shape

TensorShape([1, 1000])

In [15]:
model.summary()

Model: "tf_swin_for_image_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 swin (TFSwinMainLayer)      multiple                  27548166  
                                                                 
 classifier (Dense)          multiple                  769000    
                                                                 
Total params: 28,317,166
Trainable params: 28,288,354
Non-trainable params: 28,812
_________________________________________________________________


In [6]:
default_config = SwinConfig()
custom_config = SwinConfig(
    image_size=84,
    patch_size=3,
    num_channels=4,
    embed_dim=96,
    depths=[2, 3, 2],
    num_heads=[3, 3, 6],
    window_size=7,
    mlp_ratio=4.0,
    drop_path_rate=0.1,
)
# custom_config.save_pretrained(save_directory="./swin_config") # Save the config in a json file
swin_model = TFSwinModel(config=custom_config)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [58]:
input_layer = tf.keras.layers.Input(shape=(4, 84, 84))
conv = tf.keras.layers.Conv2D(32, 8, strides=4, activation="relu", data_format="channels_first"),


print(conv)
print(issubclass(tf.keras.layers.Conv2D, tf.keras.layers.Layer))
print(type(model))

(<keras.layers.convolutional.conv2d.Conv2D object at 0x7fbf3c322760>,)
True
<class 'transformers.models.swin.modeling_tf_swin.TFSwinModel'>


In [94]:
import gymnasium as gym
from gymnasium.wrappers import AtariPreprocessing, FrameStack


def make_env(env_name="ALE/Pong-v5", seed=42):
    env = gym.make(env_name, render_mode="rgb_array", full_action_space=False, frameskip=1)
    env = AtariPreprocessing(env)
    # env = RecordEpisodeStatistics(env)
    env = FrameStack(env, 4)
    env.observation_space.seed(seed)
    env.action_space.seed(seed)

    return env

env = make_env()
state, _ = env.reset()
state = np.array(state)
state = np.expand_dims(state, 0)
# state

array([[[[ 52,  52,  52, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         ...,
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236]],

        [[ 52,  52,  52, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         ...,
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236]],

        [[ 52,  52,  52, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         ...,
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236],
         [236, 236, 236, ..., 236, 236, 236]],

        [[ 52,  52,  52, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  87,  87,  87],
         [ 87,  87,  87, ...,  8

In [ ]:
preprocess = tf.keras.Sequential([
    #tf.keras.layers.Input(shape=(4, 84, 84)),
    tf.keras.layers.Rescaling(scale=1.0 / 255),
])
state = preprocess(state)
state

In [113]:
class SwinTransformerAtariBlock(tf.keras.Model):
    def __init__(self, num_actions):
        super(SwinTransformerAtariBlock, self).__init__()

        # Preprocessing phase
        self.rescaling = tf.keras.layers.Rescaling(scale=1.0 / 255)
        self.swin = TFSwinModel(config=configuration)
        self.action_outputs = tf.keras.layers.Dense(num_actions, name="action", activation="linear")

    def call(self, inputs, training=None, **kwargs):
        swin_outputs = self.swin(pixel_values=inputs, training=training)
        pooled_output = swin_outputs[1]
        logits = self.action_outputs(pooled_output)


        return logits


In [114]:
model = SwinTransformerAtariBlock(6)
model(state)

# inputs = tf.keras.layers.Input(shape=(4, 84, 84))
# x = tf.keras.layers.Rescaling(scale=1.0 / 255)(inputs)
# x = TFSwinModel(config=configuration)(x)
#action = tf.keras.layers.Dense(6, activation="linear")(x.pooler_output)
#rl_model = tf.keras.Model(inputs=inputs, outputs=action)

# model.summary()

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[-0.28238356,  0.78094923,  0.49464095,  0.3961748 , -1.7232966 ,
        -1.5269284 ]], dtype=float32)>